**2ª Parte – Desenvolvimento Pipeline SSIS**

In [7]:
--Usando BD local DW_47
use DW_47

Commands completed successfully.

Total execution time: 00:00:00.003

**CRIANDO UMA TABELA DE LOG**

In [7]:

IF NOT EXISTS
 (SELECT * FROM sysobjects WHERE name='LOG_CARGAS' AND xtype='U')

CREATE TABLE LOG_CARGAS
 (ID INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
 NUMERO_ERRO INT,
 SERVERIDADE_ERRO INT,
 ESTADO_ERRO  INT,
 PROC_ERRO NVARCHAR(255),
 LINHA_ERRO INT,
 MSG_ERRO NVARCHAR(255),
 SITUACAO NVARCHAR(10),
 PROCESSO NVARCHAR(30),
 DATA_INI DATETIME,
 DATA_FIM DATETIME  
 )
 GO

Commands completed successfully.

Total execution time: 00:00:00.092

**CRIANDO TABELA STAGING VENDAS**

In [1]:
-- CRIANDO TABELA STAGING
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='STG_VENDAS' AND xtype='U')
CREATE TABLE [dbo].[STG_VENDAS](
	[Segmento] [varchar](50) NULL,
	[Pais] [varchar](50) NULL,
	[Produto] [varchar](50) NULL,
	[TipoDesconto] [varchar](50) NULL,
	[QtdVendas] [varchar](50) NULL,
	[PrecoCusto] [varchar](50) NULL,
	[PrecoVenda] [varchar](50) NULL,
	[VendaBruta] [varchar](50) NULL,
	[Vendas] [varchar](50) NULL,
	[CustoVenda][varchar](50) NULL,
	[Desconto][varchar](50) NULL,
	[Lucro] [varchar](50)NULL,
	[DataVenda][varchar](50) NULL
) ON [PRIMARY]
GO


Commands completed successfully.

Total execution time: 00:00:00.159

TRUCANDO TABELA STAGING VENDAS

In [4]:
TRUNCATE TABLE STG_VENDAS; --Limpeza da tabela antes de iniciar o processo.


Commands completed successfully.

Total execution time: 00:00:00.065

**CRIANDO TABELAS DIMENSÕES**

In [3]:
--CRIANDO DIMENSAO  SEGMENTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Segmento' AND xtype='U')
CREATE TABLE [dbo].[D_Segmento](
	[Id_Segmento] [int] IDENTITY(1,1) NOT NULL,
	[NOME_SEGMENTO] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[Id_Segmento] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO PAIS
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Pais' AND xtype='U')
CREATE TABLE [dbo].[D_Pais](
	[id_Pais] [int] IDENTITY(1,1) NOT NULL,
	[NOME_PAIS] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[id_Pais] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO PRODUTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_Produto' AND xtype='U')

CREATE TABLE [dbo].[D_Produto](
	[id_Produto] [int] IDENTITY(1,1) NOT NULL,
	[NOME_PRODUTO] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[id_Produto] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO

--CRIANDO DIMENSAO TIPO DESCONTO
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='D_TipoDesconto' AND xtype='U')
CREATE TABLE [dbo].[D_TipoDesconto](
	[id_TipoDesconto] [int] IDENTITY(1,1) NOT NULL,
	[TIPO_DESCONTO] [varchar](50) NULL,
PRIMARY KEY CLUSTERED 
(
	[id_TipoDesconto] ASC
)WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY]
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.135

**CRIANDO TABELA FATO**

In [6]:
IF NOT EXISTS
  (SELECT * FROM sysobjects WHERE name='FATO_VENDA' AND xtype='U')
CREATE TABLE FATO_VENDA
(
id_Segmento  [int] NULL,
id_Pais  [int] NULL,
id_Produto  [int] NULL,
id_TipoDesconto  [int] NULL,
QtdVendas DECIMAL(10,2) NULL,
PrecoCusto DECIMAL(10,2) NULL,
PrecoVenda DECIMAL(10,2) NULL,
VendaBruta DECIMAL(10,2) NULL,
Vendas DECIMAL(10,2) NULL,
CustoVenda DECIMAL(10,2) NULL,
Desconto DECIMAL(10,2) NULL,
Lucro DECIMAL(10,2) NULL,
DataVenda DATE
)

Commands completed successfully.

Total execution time: 00:00:01.362

**CARGA TABELA DIMENSÃO SEGMENTO**

In [4]:
--Alimentando tabela dimensão segmento

BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  

MERGE D_Segmento AS Destino

USING (SELECT DISTINCT Segmento FROM STG_VENDAS) AS Origem

ON Destino.NOME_SEGMENTO = Origem.Segmento

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_SEGMENTO = Origem.Segmento

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_SEGMENTO) VALUES (Origem.Segmento);

   END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA SEGMENTO',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
       'CARGA SEGMENTO',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

 


(5 rows affected)

(1 row affected)

Total execution time: 00:00:00.068

**CARGA DIMENSÃO PAIS**

In [ ]:
--Alimentando tabela pais
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  


MERGE D_Pais AS Destino

USING (SELECT DISTINCT Pais FROM STG_VENDAS) AS Origem

ON Destino.NOME_PAIS = Origem.Pais

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_PAIS = Origem.Pais

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_PAIS) VALUES (Origem.Pais);

END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA PAIS',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA PAIS',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  



**CARGA DIMENSÃO PRODUTO**

In [ ]:
--Alimentando tabela produto
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  


MERGE D_Produto AS Destino

USING (SELECT DISTINCT Produto FROM STG_VENDAS) AS Origem

ON Destino.NOME_PRODUTO = Origem.Produto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET NOME_PRODUTO = Origem.Produto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (NOME_PRODUTO) VALUES (Origem.Produto);

END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA PRODUTO',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA PAIS',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  


**CARGA DIMENSÃO TIPO DE DESCONTO**

In [ ]:
--Alimentando tabela tipo de desconto
BEGIN TRANSACTION;  
	DECLARE @VDATA_INI DATETIME;
	SET @VDATA_INI= GETDATE();
BEGIN TRY  


MERGE D_TipoDesconto AS Destino

USING (SELECT DISTINCT TipoDesconto FROM STG_VENDAS) AS Origem

ON Destino.TIPO_DESCONTO = Origem.TipoDesconto

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET TIPO_DESCONTO = Origem.TipoDesconto

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
    INSERT (TIPO_DESCONTO) VALUES (Origem.TipoDesconto);

END TRY  
BEGIN CATCH
 	IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        'CARGA TIPO DESCONTO',
		@VDATA_INI,
		GETDATE()
		)

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
		   [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        'CARGA TIPO DESCONTO',
		@VDATA_INI,
		GETDATE()
		)

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

**CARGA FATO**

In [8]:
--alimentando tabela fato
BEGIN TRANSACTION;  
   DECLARE @VDATA_INI DATETIME;
    DECLARE @VPROCESSO VARCHAR(30);
    SET @VDATA_INI= GETDATE();
    SET @VPROCESSO= 'CARGA FATO'
BEGIN TRY  


MERGE FATO_VENDA AS Destino

USING (
SELECT b.id_Segmento,
       c.id_Pais,
       d.id_Produto,
       e.id_TipoDesconto,
       CAST(REPLACE(a.QtdVendas,',','.') AS decimal(10,2)) AS QtdVendas,
       CAST(REPLACE(a.PrecoCusto,',','.') AS decimal(10,2))AS PrecoCusto,
       CAST(REPLACE(a.PrecoVenda,',','.') AS decimal(10,2)) AS PrecoVenda,
       CAST(REPLACE(a.VendaBruta,',','.') AS decimal(10,2))AS VendaBruta,
       CAST(REPLACE(a.Vendas,',','.') AS decimal(10,2)) AS Vendas,
       CAST(REPLACE(a.CustoVenda,',','.') AS decimal(10,2))AS CustoVenda,
       CAST(REPLACE(a.Desconto,',','.') AS decimal(10,2)) AS Desconto,
       CAST(REPLACE(a.Lucro,',','.') AS decimal(10,2))AS Lucro,
       CAST(a.DataVenda AS DATE) AS DataVenda
  FROM STG_VENDAS a
   LEFT JOIN D_Segmento b
	ON a.Segmento=B.NOME_SEGMENTO

  LEFT JOIN D_Pais c
	ON a.Pais=c.NOME_PAIS

  LEFT JOIN D_Produto d
	ON a.Produto=D.NOME_PRODUTO

  LEFT JOIN D_TipoDesconto e
	ON a.TipoDesconto=E.TIPO_DESCONTO

) AS origem
    ON Destino.DataVenda = Origem.DataVenda

-- Há registro no destino e na origem
WHEN MATCHED 

THEN 
    UPDATE SET  id_Segmento=origem.id_Segmento, 
                id_Pais=origem.id_Pais,
                id_Produto=origem.id_Produto,
                id_TipoDesconto=origem.id_TipoDesconto,
                QtdVendas=origem.QtdVendas,
                PrecoCusto=origem.PrecoCusto,
                PrecoVenda=origem.PrecoVenda,
                VendaBruta=origem.VendaBruta,
                Vendas=origem.Vendas,
                CustoVenda=origem.CustoVenda,
                Desconto=origem.Desconto,
                Lucro=origem.Lucro,
                DataVenda=origem.DataVenda

--Quando não há registro no destino e há na origem
WHEN NOT MATCHED 

THEN 
   INSERT  
           (id_Segmento, id_Pais, id_Produto , id_TipoDesconto, QtdVendas, 
           PrecoCusto, PrecoVenda, VendaBruta, Vendas, CustoVenda, Desconto, Lucro, DataVenda)
     VALUES
           (origem.id_Segmento, 
           origem.id_Pais,
           origem.id_Produto,
           origem.id_TipoDesconto,
           origem.QtdVendas,
           origem.PrecoCusto,
           origem.PrecoVenda,
           origem.VendaBruta,
           origem.Vendas,
           origem.CustoVenda,
           origem.Desconto,
           origem.Lucro,
           origem.DataVenda);

END TRY  
BEGIN CATCH
   IF @@TRANCOUNT > 0  
        ROLLBACK TRANSACTION; 

INSERT INTO [dbo].[LOG_CARGAS]
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
         [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'ERRO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

        
END CATCH;  

INSERT INTO LOG_CARGAS
           ([NUMERO_ERRO],
           [SERVERIDADE_ERRO],
           [ESTADO_ERRO],
           [PROC_ERRO],
           [LINHA_ERRO],
           [MSG_ERRO],
           [SITUACAO],
           [PROCESSO],
           [DATA_INI],
           [DATA_FIM])
     VALUES
        (ERROR_NUMBER(),
        ERROR_SEVERITY(),
        ERROR_STATE(),
        ERROR_PROCEDURE(),
        ERROR_LINE() ,
        ERROR_MESSAGE() ,
        'SUCESSO',
        @VPROCESSO,
      @VDATA_INI,
      GETDATE()
      )

IF @@TRANCOUNT > 0  
    COMMIT TRANSACTION;  
GO  

(0 rows affected)

(1 row affected)

Total execution time: 00:00:00.050

**Fim do processo**

TRUNCANDO OS DADOS DAS TABELAS

In [6]:
--Limpeza das tabelas para iniciar um novo processo

/*
TRUNCATE TABLE D_SEGMENTO;
TRUNCATE TABLE D_PAIS;
TRUNCATE TABLE D_PRODUTO;
TRUNCATE TABLE D_TIPODESCONTO;
TRUNCATE TABLE FATO_VENDA;
TRUNCATE TABLE STG_VENDAS;
TRUNCATE TABLE LOG_CARGAS; */

: Msg 102, Level 15, State 1, Line 1
Sintaxe incorreta próxima a '/'.

: Msg 102, Level 15, State 1, Line 8
Sintaxe incorreta próxima a '*'.

Total execution time: 00:00:00.093